In [2]:
from xml.dom import minidom

In [40]:
def string2coord(string):
    '''take a string such as "x,y" and return a list of two floats [x,y]'''
    return [float(x) for x in string.split(',')]

def d_bezier_to_straight_path(path_d):
    '''
    Input is the d attribute of an svg path element representing a relative bezier curves.
    Output is the list of points of this path (first point is absolute, others relative to previous ones), 
    without curve parameters.
    '''
    path_d = path_d.split(' ')
    points = list()
    points.append(string2coord(path_d[1]))
    
    for i in range(5, len(path_d), 3):
        points.append(string2coord(path_d[i]))      
    return points

def d_straigh_to_straight_path(path_d):
    '''
    Input is the d attribute of an svg path element representing a relative straight path.
    Output is the list of points of this path (first point is absolute, others relative to previous ones)
    '''
    path_d = path_d.lstrip('m ').rstrip('z ').split(' ')
    points = list()
    for pt in path_d:
        points.append(string2coord(pt))
    return points

def d_qbezier_to_straight_path(path_d):
    '''
    Input is the d attribute of an svg path element representing an absolute quadratic beizer path.
    Typically produced by the pen tool of Concept (tested on Android).
    Output is the list of absolute points of this path.
    '''
    d_path2 = list()
    for i in allpath[0].getAttribute('d').split(' '):
        try:
            d_path2.append(float(i))
        except:
            pass
    
    points = list()
    for i in range(0, len(d_path2), 4):
        points.append([d_path2[i], d_path2[i+1]])    
    return points

def relative_path_to_absolute(points):    
    x = 0
    y = 0
    abs_path=list()
    for pt in points:
        x = pt[0] + x
        y = pt[1] + y
        abs_path.append([x,y])
    return abs_path

In [45]:
def translate_scale_path(viewBox,W_ratio, H_ratio, points):
    ts_points = list()
    for pt in points:
        x = (pt[0] - viewBox[0])*W_ratio
        y = (pt[1] - viewBox[1])*H_ratio
        ts_points.append([x,y])      
    return ts_points
    

In [46]:
doc = minidom.parse('./example/test3.svg')

# Collect svg viewbox
viewBox = doc.getElementsByTagName('svg')[0].getAttribute('viewBox')
viewBox = [float(n) for n in viewBox.split(' ')]
viewBox_W, viewBox_H = viewBox[2] - viewBox[0], viewBox[3] - viewBox[1]
W = doc.getElementsByTagName('svg')[0].getAttribute('width')
W = float(W.replace('px', ''))
H = doc.getElementsByTagName('svg')[0].getAttribute('height').replace('px', '')
H = float(H.replace('px', ''))
# Collect all path
allpath = [path for path in doc.getElementsByTagName('path')]

W_ratio = W / viewBox_W
H_ratio = H / viewBox_H

In [47]:
# Dealing with relative bezier curves paths
paths = list()
for path in allpath:
    d = path.getAttribute('d')

    if 'c' in d:
        print('Found c command, assuming relative Bezier curves')
        points = d_bezier_to_straight_path(d)
        paths.append(relative_path_to_absolute(points))

        
    elif 'Q' in d:
        print('Found Q command, assuming Q Bezier curves')
        points = d_qbezier_to_straight_path(d)
        paths.append(points)
  
    else:
        print('No command found, assuming straight relative path')
        points = d_straigh_to_straight_path(d)
        paths.append(relative_path_to_absolute(points))

ts_paths = list()
for p in paths:
    ts_paths.append(translate_scale_path(viewBox, W_ratio, H_ratio, p))


Found Q command, assuming Q Bezier curves
Found Q command, assuming Q Bezier curves
Found Q command, assuming Q Bezier curves
Found Q command, assuming Q Bezier curves
Found Q command, assuming Q Bezier curves
Found Q command, assuming Q Bezier curves
Found Q command, assuming Q Bezier curves
Found Q command, assuming Q Bezier curves


In [48]:
ts_paths

[[[2841.420098812003, 1567.065697818152],
  [2841.5057292733845, 1568.0220980191607],
  [2841.240909142814, 1568.2939379103186],
  [2840.9237592858444, 1567.3077742175774],
  [2839.4949991801936, 1564.2500715002448],
  [2837.181390973596, 1560.5137611714085],
  [2835.2420195982236, 1558.0037067019564],
  [2833.954391178925, 1556.0155054542167],
  [2832.9220683944873, 1554.2058851568727],
  [2832.09272151851, 1552.888354589508],
  [2831.4346355652974, 1551.7355153430638],
  [2830.904995304157, 1550.71958815859],
  [2830.863765822751, 1550.5112437164616],
  [2830.5783309514777, 1550.1739241434918],
  [2828.9751384244937, 1546.9158139151598],
  [2827.14994099763, 1542.5505017943735],
  [2826.3951243380416, 1540.238870603139],
  [2826.1525046974593, 1539.2011168580611],
  [2825.922571051156, 1538.1554261818546],
  [2825.7338668862585, 1537.3180799477768],
  [2825.6799514105737, 1537.1613255579848],
  [2825.339015314331, 1536.1830987963722],
  [2824.208376074232, 1534.0282219949295],
  [282